In [32]:
pip install nhlpy


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

from nhlpy.constants import BASE_URL
from nhlpy import team,game,schedule #There are other modules but this should do it for now

In [2]:
# Function that takes a game object created by nhlpy.game and 
# returns a 0 for a home team win and 1 for an away team win.
# Somehow, I could not find this data more directly using this
# package.

## Changed: add teams' names and abbrevs information 

def whoWon(game):
    scoreData = game.all_stats()['liveData']['linescore']
    periodData = scoreData['periods']
    homegoals = 0
    awaygoals = 0
    
    # changed: 3 -> len(periodData) 
    # since not all games has 3 periods, eg.2019120001
    
    for i in range(len(periodData)):
        homegoals = homegoals + periodData[i]['home']['goals']
        awaygoals = awaygoals + periodData[i]['away']['goals']
    if homegoals == awaygoals:
        shootoutData = scoreData['shootoutInfo'] 
        homeshootout = shootoutData['home']['scores']
        awayshootout = shootoutData['away']['scores']
        winner = homeshootout < awayshootout
    else:
        winner = homegoals < awaygoals
    
    home_name = game.all_stats()['gameData']['teams']['home']['name']
    away_name = game.all_stats()['gameData']['teams']['away']['name']
    
    home_abbrev = game.all_stats()['gameData']['teams']['home']['abbreviation']
    away_abbrev = game.all_stats()['gameData']['teams']['away']['abbreviation']
    
    
    winner_dict = {'Teams': [home_name, away_name], 
                   'Abbrevs': [home_abbrev, away_abbrev], 
                   'GamePK': game.all_stats()['gameData']['game']['pk'], 
                   'Winner': int(winner)}
    # return int(winner)
    return winner_dict

# A few notes: this will not work to determine ties (which happened prior to 2004),
# nor will it work in playoff games that go into overtime. I think it's okay for 
# now but I will need to fix if we include those games.

In [3]:
# Example game: Golden Knights @ Blue Jackets 1-4 (home team won)
exgame = game.Game(2017021023) # This number is the GamePk, the numerical code associated with a game
whoWon(exgame)

{'Teams': ['Columbus Blue Jackets', 'Vegas Golden Knights'],
 'Abbrevs': ['CBJ', 'VGK'],
 'GamePK': 2017021023,
 'Winner': 0}

In [89]:
# This function takes a schedule object, and returns a dictionary
# with all of the information for the games in that season

# This was a method in nhlpy that is on Github, but not PyPI
# Defining it here as a function to make life easier
def season(schedule, seasonid=None):
    response = requests.get("{0}/schedule{1}{2}".format(BASE_URL, "?season=", seasonid))
    schedule.data = response.json()
    #del schedule.data["copyright"]
    return schedule.data

In [84]:
# Example season data:
thisschedule = schedule.Schedule()
thisseason = season(thisschedule,20202021)
print(thisseason.keys())
print(thisseason['totalGames'])
print(thisseason['dates'][0]['games'][4].keys())
print(thisseason['dates'][0]['games'][4]['gamePk'])


dict_keys(['totalItems', 'totalEvents', 'totalGames', 'totalMatches', 'metaData', 'wait', 'dates'])
952
dict_keys(['gamePk', 'link', 'gameType', 'season', 'gameDate', 'status', 'teams', 'venue', 'content'])
2020020005


In [85]:
# This is a function that gets all of the game pks from a given season
# Let's see if this is faster than hockey-scraper

def getPkList(season):
    numGames = season['totalGames']
    pklist = np.zeros(numGames)
    i = 0
    for date in season['dates']:
        for game in date['games']:
            pklist[i] = game['gamePk']
            i = i + 1
    return pklist

# In the future, it might make more sense to just grab everything we 
# need while looping through the season, but this should be a useful 
# tool for now.


In [87]:
# Example Pk List:
expklist = getPkList(thisseason)
expklist

array([2.02002000e+09, 2.02002000e+09, 2.02002000e+09, 2.02002000e+09,
       2.02002000e+09, 2.02002001e+09, 2.02002001e+09, 2.02002001e+09,
       2.02002001e+09, 2.02002001e+09, 2.02002001e+09, 2.02002001e+09,
       2.02002001e+09, 2.02002002e+09, 2.02002002e+09, 2.02002002e+09,
       2.02002002e+09, 2.02002002e+09, 2.02002002e+09, 2.02002002e+09,
       2.02002002e+09, 2.02002003e+09, 2.02002002e+09, 2.02002002e+09,
       2.02002003e+09, 2.02002003e+09, 2.02002003e+09, 2.02002003e+09,
       2.02002003e+09, 2.02002003e+09, 2.02002004e+09, 2.02002003e+09,
       2.02002004e+09, 2.02002004e+09, 2.02002004e+09, 2.02002004e+09,
       2.02002004e+09, 2.02002004e+09, 2.02002004e+09, 2.02002004e+09,
       2.02002004e+09, 2.02002004e+09, 2.02002005e+09, 2.02002005e+09,
       2.02002005e+09, 2.02002005e+09, 2.02002005e+09, 2.02002005e+09,
       2.02002005e+09, 2.02002006e+09, 2.02002006e+09, 2.02002006e+09,
       2.02002006e+09, 2.02002006e+09, 2.02002006e+09, 2.02002006e+09,
      

In [93]:
# Win list for a season: perhaps this is what we regress onto?

winlist = np.zeros(len(expklist))
for i, pk in enumerate(expklist):
    thisgame = game.Game(int(pk))
    winlist[i] = whoWon(thisgame)
    
# This loop takes me about one minute

### create winner_hone_or_away csv dataset

In [ ]:
a = 20202021
for i in range(10):
    a = a-1
    a = a-10000
    print('creating season data: ', a)
    filename = str(a)+'_winner.json'

    thisschedule = schedule.Schedule()
    thisseason = season(thisschedule, a)
    expklist = getPkList(thisseason)

    season_stats = []
    for i, pk in enumerate(expklist):
        thisgame = game.Game(int(pk))
        season_stats.append(whoWon(thisgame))

    with open(filename, 'w') as outfile:
        json.dump(season_stats, outfile)
    outfile.close()



In [94]:
i

951

In [92]:
pk

2020020001.0

In [95]:
winlist

array([0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
       1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 1., 1., 1., 0.